In [1]:
from random import sample
import numpy as np
import pandas as pd
import itertools
pd.set_option('display.expand_frame_repr', False)

%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
import re

#nltk.download('stopwords')
#nltk.download('punkt')
ps = PorterStemmer()


def itemParser(s):
    
    s = s.replace("\n", "")
    s = re.sub(r'[^\w\s]', '', s)
    s = re.sub(r"(\d)", "", s)
    s = re.sub(r'\([^)]*\)', '', s)
    s = re.sub(u'\w*\u2122', '', s)

    s = s.lower()

    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(s)

    filtered_sentence = [ps.stem(w) for w in word_tokens if not w in stop_words]
    s = ' '.join(filtered_sentence)
    
    return s

In [3]:
superm = []
baskets = []
with open('./data/groceries.csv', 'r') as f:
    for line in f:
        basket = []
        items = line.split(",")
        for item in items:
            parsed = itemParser(item)
            superm.append(parsed)
            basket.append(parsed)
        baskets.append(basket)

In [4]:
receipts = pd.read_csv('./data/train-clean.csv')

In [8]:
def jaccard_sim(s1, s2):
    return len(s1.intersection(s2))/len(s1.union(s2))

In [32]:
def match(basket):

    valid = []
    for row in receipts.itertuples(index=True, name='Pandas'):

        c = getattr(row, "cuisine")    
        x = getattr(row, "new_ingredients")
        x = x.replace("', '", ',')    
        x = x[2:-2]
        x = set(x.split(","))
        b = set(basket)
        j = jaccard_sim(x, b)

        if j > 0:
            valid.append((j, c))
            
        valid = sorted(valid, key = lambda x: x[0], reverse=True)
            

    tmp = {}
    for v in valid:
        if v[1] not in tmp:
            tmp[v[1]] = 0
        tmp[v[1]] += v[0]
        
    total = 0
    for k in tmp:
        total += tmp[k]
            
    for k in tmp:
        tmp[k] = 100*(tmp[k]/total)
        
    out = []
    for i in tmp:
        out.append([i, tmp[i]])

    out = sorted(out, key = lambda x: x[1], reverse=True)

    return out
    

In [52]:
match(baskets[1])

[['italian', 24.00701101670278],
 ['vietnamese', 14.347689704385902],
 ['french', 10.162946873940014],
 ['southern_us', 8.480004142929678],
 ['mexican', 8.40698979595051],
 ['cajun_creole', 8.048342468472669],
 ['irish', 7.781546585539875],
 ['japanese', 4.114787512656692],
 ['greek', 4.001938243991935],
 ['british', 3.11261863421595],
 ['brazilian', 2.6284335133379133],
 ['indian', 1.562412490900555],
 ['thai', 1.3833860596515333],
 ['russian', 1.1318613215330726],
 ['jamaican', 0.8300316357909199]]

In [100]:
final = []

In [97]:
i = 0
for b in baskets[:600]:
    
    m = match(b)
    if len(m) == 0:
        final.append((b, "None"))
    else:
        if len(m) == 1:
            final.append((b, (m[0][0], m[0][1])))
        elif len(m) == 2:
            final.append((b, (m[0][0], m[0][1]), (m[1][0], m[1][1])))
        else:
            final.append((b, (m[0][0], m[0][1]), (m[1][0], m[1][1]), (m[2][0], m[2][1])))
            
    i += 1
    if i%5 == 0:
        print(i)

10
20
30
40


KeyboardInterrupt: 

In [99]:
len(final)

45

In [197]:
from multiprocessing import Pool

def f(b):
    m = match(b[0])
    if len(m) == 0:
        ret = (b[0], "None")
    else:
        if len(m) == 1:
            ret = (b[0], (m[0][0], m[0][1]))
        elif len(m) == 2:
            ret = (b[0], (m[0][0], m[0][1]), (m[1][0], m[1][1]))
        else:
            ret = (b[0], (m[0][0], m[0][1]), (m[1][0], m[1][1]), (m[2][0], m[2][1]))
    return ret

'''
num = list(range(9835))
Z = list(zip(baskets, num))'''

with Pool(6) as p:
    ris = p.map(f, Z[9200:])
    for r in ris:
        valid.append(r)

In [198]:
len(valid)

9835

In [254]:
newVect = []
leng = 10
for v in valid:
    bas = v[0]
    if isinstance(bas, tuple):
        bas = bas[0]
    
    itemrow = []
    itemrow.append(bas)
    
    missing = 32 - len(bas)
    for _ in range(missing):
        bas.append("")
    
    itemrow.append(v[1])
    
    if len(v) >= 3:
        itemrow.append(v[2])
    else:        
        itemrow.append("None")
        
    if len(v) >= 4:
        itemrow.append(v[3])
    else:        
        itemrow.append("None")
    
    newVect.append(itemrow)
    

In [261]:
output = pd.DataFrame(newVect)
output.columns = ["Basket", "Cuisine 1", "Cuisine 2", "Cuisine 3"]

output

,Basket,Cuisine 1,Cuisine 2,Cuisine 3
0,"[citru fruit, semifinish bread, margarin, read...","(southern_us, 26.2716193710661)","(italian, 15.472477917954564)","(irish, 10.5824849581435)"
1,"[tropic fruit, yogurt, coffe, , , , , , , , , ...","(italian, 24.00701101670278)","(vietnamese, 14.347689704385902)","(french, 10.162946873940014)"
2,"[whole milk, , , , , , , , , , , , , , , , , ,...","(southern_us, 23.582198352735574)","(italian, 20.153301317917553)","(french, 16.226102444360386)"
3,"[pip fruit, yogurt, cream chees, meat spread, ...","(mexican, 33.228198692510745)","(southern_us, 20.261089748334847)","(italian, 19.004440220658868)"
4,"[veget, whole milk, condens milk, long life ba...","(southern_us, 19.32058683814756)","(italian, 18.231888936520285)","(french, 12.977334797740781)"
...,...,...,...,...
9830,"[sausag, chicken, beef, hamburg meat, citru fr...","(southern_us, 22.659456570440092)","(italian, 18.005963859911034)","(mexican, 10.033571663673774)"
9831,"[cook chocol, , , , , , , , , , , , , , , , , ...",None,None,None
9832,"[chicken, citru fruit, veget, butter, yogurt, ...","(southern_us, 23.841784480144742)","(italian, 19.309751101751697)","(mexican, 10.013761140826167)"
9833,"[semifinish bread, bottl water, soda, bottl be...","(southern_us, 46.82311299507751)","(indian, 30.526949791159375)","(chinese, 8.45553517169925)"


In [200]:
import json
with open('output.txt', 'w') as out_file:
    json.dump(valid, out_file)

In [201]:
with open('output.txt', 'r') as in_file:
    your_list = json.load(in_file)

In [202]:
len(your_list)

9835

In [203]:
your_list[0]

[[['citru fruit', 'semifinish bread', 'margarin', 'readi soup'], 0],
 ['southern_us', 26.2716193710661],
 ['italian', 15.472477917954564],
 ['irish', 10.5824849581435]]

In [264]:
output.to_pickle("./data/basket-receipts.pkl")

In [265]:
unpickled_df = pd.read_pickle("./data/basket-receipts.pkl")
unpickled_df

,Basket,Cuisine 1,Cuisine 2,Cuisine 3
0,"[citru fruit, semifinish bread, margarin, read...","(southern_us, 26.2716193710661)","(italian, 15.472477917954564)","(irish, 10.5824849581435)"
1,"[tropic fruit, yogurt, coffe, , , , , , , , , ...","(italian, 24.00701101670278)","(vietnamese, 14.347689704385902)","(french, 10.162946873940014)"
2,"[whole milk, , , , , , , , , , , , , , , , , ,...","(southern_us, 23.582198352735574)","(italian, 20.153301317917553)","(french, 16.226102444360386)"
3,"[pip fruit, yogurt, cream chees, meat spread, ...","(mexican, 33.228198692510745)","(southern_us, 20.261089748334847)","(italian, 19.004440220658868)"
4,"[veget, whole milk, condens milk, long life ba...","(southern_us, 19.32058683814756)","(italian, 18.231888936520285)","(french, 12.977334797740781)"
...,...,...,...,...
9830,"[sausag, chicken, beef, hamburg meat, citru fr...","(southern_us, 22.659456570440092)","(italian, 18.005963859911034)","(mexican, 10.033571663673774)"
9831,"[cook chocol, , , , , , , , , , , , , , , , , ...",None,None,None
9832,"[chicken, citru fruit, veget, butter, yogurt, ...","(southern_us, 23.841784480144742)","(italian, 19.309751101751697)","(mexican, 10.013761140826167)"
9833,"[semifinish bread, bottl water, soda, bottl be...","(southern_us, 46.82311299507751)","(indian, 30.526949791159375)","(chinese, 8.45553517169925)"
